In [1]:
import torch

def mask_generator(M,S,N):
    """ Generate dropout-masks according to the masksemble schema.
        Schmea is slightly modified, to ensure that no channel is completly dropped

    Args:
        M (_type_): toal number of ones per mask
        S (_type_): scale that controls the amount of overlap given N and M
        N (_type_): number of generated masks
    """
    zero_rows = 1
    while(zero_rows > 0):
        # 1 create N vectors of all zeros of size MxS
        zero_mask = torch.zeros(size=(int(S*M),N))
        #print("zeros mask")
        #print(zero_mask)

        #2 set M elements randomly to one for each vector
        for n in range(N):
            one_mask = torch.randperm(int(S*M))[:M]
            zero_mask[one_mask,n] = 1
        #print("mask filled with ones")
        #print(zero_mask)

        #3 check for all zero rows
        zero_rows =  torch.sum(zero_mask.sum(dim=1) == 0)
        #print(f"found {zero_rows} zero rows")
        
    return zero_mask


In [45]:
from torch import nn
import random
from typing import List,Tuple, Union
class MasksembleModel(nn.Module):
    def __init__(self, neurons_per_layer: List[int], M: int, S: int, N: int) -> None:
        super().__init__()
        self.mode = "training"
        for num in neurons_per_layer:
            if num != M * S:
                raise ValueError(f"Masksemble parameter and model parameter do not fit! Number of neurons: {num} != M*S = {M*S}. "
                                 "Number of neurons must be equal to M*S for each layer")
                
        self.layer = [nn.Linear(2, neurons_per_layer[0])]
        self.activation = nn.Tanh()
        for i in range(1, len(neurons_per_layer)):
            self.layer.append(nn.Linear(neurons_per_layer[i-1], neurons_per_layer[i]))
        self.output_layer = nn.Linear(neurons_per_layer[-1], 2)
        
        self.masks = list(mask_generator(M, S, N).transpose(0, 1))
                    
        self.r = nn.Parameter(torch.tensor([0], dtype=torch.float, requires_grad=True))
        
    def forward(self, x: torch.Tensor) -> Union[Tuple[List[torch.Tensor], torch.Tensor], Tuple[torch.Tensor, torch.Tensor]]:
        if self.mode == "training":
            mask = random.choice(self.masks)
            y = self.forward_with_mask(mask, x)
        else:
            predictions = []
            for mask in self.masks:
                predictions.append(self.forward_with_mask(mask, x))
            y = torch.stack(predictions)
            if self.mode == "eval":
                y = torch.mean(y, dim=0)
            if self.mode == "uq":
                y = [torch.mean(y, dim=0), torch.var(y, dim=0)]
        return y, self.r
        
    def forward_with_mask(self, mask: torch.Tensor, x: torch.Tensor) -> torch.Tensor:
        batch_size = x.size(0)
        mask = mask.unsqueeze(0).expand(batch_size, -1)  # Expand mask to match batch size
        print(mask)
        for layer in self.layer:
            x = layer(x)
            x = self.activation(x)
            x = x * mask
        return self.output_layer(x)
    
    def set_mode(self, mode: str) -> None:
        """Set the mode of the model, this will influence the behavior of the forward pass.
        
        Args:
            mode (str): One of 'training', 'eval', 'uq'.
        """
        if mode in ["training", "eval", "uq"]:
            self.mode = mode
        else:
            raise ValueError(f"Unsupported mode {mode}. Only 'training', 'eval', and 'uq' are available.")
    
    
    
    

In [46]:
masksemble = MasksembleModel([20,20,20],16,1.25,4)
input = torch.tensor([[10,10],[9,9]],dtype=torch.float32)
masksemble.set_mode("training")

In [47]:
pred,r = masksemble(input)

tensor([1., 1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 0., 1., 0., 1.,
        1., 1.])


In [48]:
pred

tensor([[-0.1636,  0.0822],
        [-0.1693,  0.0746]], grad_fn=<AddmmBackward0>)

In [28]:
torch.mean(pred.detach(),dim=0)

tensor([[-0.0942,  0.0741],
        [-0.0943,  0.0744]])

In [64]:
masks = list(mask_generator(2,2,4).transpose(0,1))
print(masks[0].dtype)

torch.float32


In [41]:
import random
random.choice(masks)

tensor([0., 1., 1., 0.])